In [54]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
# Import the datetime module from the datetime library.
from datetime import datetime
# Import the API key.
from config import weather_api_key
import requests
from datetime import datetime

In [65]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [66]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [67]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

593

In [68]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=96c59c82b0c62cd93888a34be01334d3


In [69]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [70]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

Processing Record 1 of Set 1 | jamestown
Processing Record 2 of Set 1 | port elizabeth
Processing Record 3 of Set 1 | barrow
Processing Record 4 of Set 1 | tuy hoa
Processing Record 5 of Set 1 | joshimath
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | ilo
Processing Record 8 of Set 1 | gat
Processing Record 9 of Set 1 | busselton
Processing Record 10 of Set 1 | norman wells
Processing Record 11 of Set 1 | puerto ayora
Processing Record 12 of Set 1 | atuona
Processing Record 13 of Set 1 | tuatapere
Processing Record 14 of Set 1 | punta arenas
Processing Record 15 of Set 1 | hilo
Processing Record 16 of Set 1 | kapaa
Processing Record 17 of Set 1 | bluff
Processing Record 18 of Set 1 | port lincoln
Processing Record 19 of Set 1 | illoqqortoormiut
Processing Record 20 of Set 1 | bredasdorp
Processing Record 21 of Set 1 | hithadhoo
Processing Record 22 of Set 1 | okhotsk
Processing Record 23 of Set 1 | saint-joseph
Processing Record 24 of Set 1 | provideniya
Processi

In [71]:
# Run an API request for each of the cities.
try:
    # Parse the JSON and retrieve data.
    city_weather = requests.get(city_url).json()
    # Parse out the needed data.
    city_lat = city_weather["coord"]["lat"]
    city_lng = city_weather["coord"]["lon"]
    city_max_temp = city_weather["main"]["temp_max"]
    city_humidity = city_weather["main"]["humidity"]
    city_clouds = city_weather["clouds"]["all"]
    city_wind = city_weather["wind"]["speed"]
    city_country = city_weather["sys"]["country"]
    # Convert the date to ISO standard.
    city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
    # Append the city information into city_data list.
    city_data.append({"City": city.title(),
                        "Lat": city_lat,
                        "Lng": city_lng,
                        "Max Temp": city_max_temp,
                        "Humidity": city_humidity,
                        "Cloudiness": city_clouds,
                        "Wind Speed": city_wind,
                        "Country": city_country,
                        "Date": city_date})

# If an error is experienced, skip the city.
except:
    print("City not found. Skipping...")
    pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Data Retrieval Complete      
-----------------------------


In [78]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Irshava,100,UA,2021-05-30 23:48:35,92,48.3167,23.0385,52.66,2.8


In [79]:
new_city_data = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp','Humidity','Cloudiness','Wind Speed']
new_city_data_df = city_data_df[new_city_data]
new_city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Irshava,UA,2021-05-30 23:48:35,48.3167,23.0385,52.66,92,100,2.8


In [80]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")